In [26]:
import requests
import base64
from typing import Tuple

YOUR_EMAIL = "alvarmaciel@gmail.com"

# URL base del desafío
BASE_URL = f"https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-decrypt/{YOUR_EMAIL}/encrypt"

print(f"URL configurada: {BASE_URL}")

URL configurada: https://ciberseguridad.diplomatura.unc.edu.ar/cripto/ecb-decrypt/alvarmaciel@gmail.com/encrypt


In [58]:
def get_message(message: str) -> bytes:
    """
    Registra un usuario y obtiene el perfil cifrado.
    
    Args:
        message: mensaje en base64
    
    Returns:
        ciphertext
    """
    
    # Hacer POST request
    response = requests.post(
        BASE_URL,
        files={
            'message': message,
        }
    )
    
    if response.status_code != 200:
        raise Exception(f"Error en la petición: {response.status_code}\n{response.text}")
    
    # Decodificar respuesta base64
    encrypted_message = base64.b64decode(response.text.strip())
    
    return encrypted_message

def response_analysis(ciphertext):
    block_size = 16
    blocks = [ciphertext[i:i+block_size] for i in range(0, len(ciphertext), block_size)]
    print(f"Total bloques: {len(blocks)}")
    
    for i, block in enumerate(blocks):
        print(f"Bloque {i}: {block.hex()}")
    return blocks 

In [59]:
cipher1 = get_message(base64.b64encode(b'A' * 15).decode())
cipher2 = get_message(base64.b64encode(b'A' * 14).decode())

print(f"15 A's, bloque 0: {cipher1[:16].hex()}")
print(f"14 A's, bloque 0: {cipher2[:16].hex()}")

15 A's, bloque 0: fff9183484a7627c68cd52682af55633
14 A's, bloque 0: 52ce51a5d82dbcfd0e4025c6c1257e4c


In [60]:
encrypted = get_message(base64.b64encode(b'A' * 15).decode())
response_analysis(encrypted)

Total bloques: 10
Bloque 0: fff9183484a7627c68cd52682af55633
Bloque 1: 60758239f561612be0b7b0f0a7595878
Bloque 2: 9b6a50d7762081c467a3a36d16d99ded
Bloque 3: de8828151c98348e55abe4e16da0d894
Bloque 4: 50ebee8a67fe998ee9b2b6b6ffba6a02
Bloque 5: 668a9e2300d0dcdb6caf5586628eb6a1
Bloque 6: 0c80791bdc74e156b2fc062fdeb711c5
Bloque 7: 2c5d40cb3b8b4b1bbb67e5f939059bd6
Bloque 8: 65a95ceff405b5ab17054bd0b4bf16fa
Bloque 9: 99926bf7ae70ca2fd166c93448fa5275


[b'\xff\xf9\x184\x84\xa7b|h\xcdRh*\xf5V3',
 b'`u\x829\xf5aa+\xe0\xb7\xb0\xf0\xa7YXx',
 b'\x9bjP\xd7v \x81\xc4g\xa3\xa3m\x16\xd9\x9d\xed',
 b'\xde\x88(\x15\x1c\x984\x8eU\xab\xe4\xe1m\xa0\xd8\x94',
 b'P\xeb\xee\x8ag\xfe\x99\x8e\xe9\xb2\xb6\xb6\xff\xbaj\x02',
 b'f\x8a\x9e#\x00\xd0\xdc\xdbl\xafU\x86b\x8e\xb6\xa1',
 b'\x0c\x80y\x1b\xdct\xe1V\xb2\xfc\x06/\xde\xb7\x11\xc5',
 b',]@\xcb;\x8bK\x1b\xbbg\xe5\xf99\x05\x9b\xd6',
 b'e\xa9\\\xef\xf4\x05\xb5\xab\x17\x05K\xd0\xb4\xbf\x16\xfa',
 b'\x99\x92k\xf7\xaep\xca/\xd1f\xc94H\xfaRu']

In [61]:
def find_secret_byte(email, known_secret, byte_position):
    """
    Encuentra un byte específico del secreto.
    
    Args:
        email: tu email
        known_secret: bytes del secreto ya descubiertos (como bytes)
        byte_position: posición del byte a descubrir (0-indexed)
    
    Returns:
        El byte encontrado o None
    """
    # Calcular cuántas 'A's necesitamos para alinear
    num_as = 15 - (byte_position % 16)
    prefix = b'A' * num_as
    
    # Determinar en qué bloque está este byte
    target_block_index = byte_position // 16
    
    # Paso 1: Obtener el bloque objetivo (con el byte desconocido)
    target_ciphertext = get_message(base64.b64encode(prefix).decode())
    blocks = [target_ciphertext[i:i+16] for i in range(0, len(target_ciphertext), 16)]
    target_block = blocks[target_block_index]
    
    print(f"Buscando byte {byte_position}, usando {num_as} A's, comparando bloque {target_block_index}")
    print(f"Target block: {target_block.hex()}")
    
    # Paso 2: Probar cada byte posible
    for byte_val in range(256):
        # Construir: prefix + conocido + candidato
        test_message = prefix + known_secret + bytes([byte_val])
        test_ciphertext = get_message(base64.b64encode(test_message).decode())
        test_blocks = [test_ciphertext[i:i+16] for i in range(0, len(test_ciphertext), 16)]
        test_block = test_blocks[target_block_index]
        
        if test_block == target_block:
            char = chr(byte_val) if 32 <= byte_val < 127 else f'\\x{byte_val:02x}'
            print(f"✓ Encontrado: {char}")
            return bytes([byte_val])
    
    print("✗ No encontrado")
    return None

In [62]:
def recover_full_secret(email, secret_length):
    """
    Recupera el secreto completo byte por byte.
    """
    known_secret = b""
    
    for position in range(secret_length):
        print(f"\n--- Byte {position + 1}/{secret_length} ---")
        byte = find_secret_byte(email, known_secret, position)
        
        if byte is None:
            print(f"Fallo en posición {position}")
            break
        
        known_secret += byte
        
        # Mostrar progreso cada 16 bytes
        if (position + 1) % 16 == 0:
            print(f"\nProgreso ({position + 1} bytes):")
            print(f"Hex: {known_secret.hex()}")
            try:
                print(f"ASCII: {known_secret.decode('ascii', errors='replace')}")
            except:
                print("ASCII: (no decodificable)")
    
    return known_secret

# Ejecutar
secret = recover_full_secret("YOUR_EMAIL", 128)
print(f"\n\n=== SECRETO COMPLETO ===")
print(f"Hex: {secret.hex()}")
print(f"ASCII: {secret}")


--- Byte 1/128 ---
Buscando byte 0, usando 15 A's, comparando bloque 0
Target block: fff9183484a7627c68cd52682af55633
✓ Encontrado: A

--- Byte 2/128 ---
Buscando byte 1, usando 14 A's, comparando bloque 0
Target block: 52ce51a5d82dbcfd0e4025c6c1257e4c
✓ Encontrado: l

--- Byte 3/128 ---
Buscando byte 2, usando 13 A's, comparando bloque 0
Target block: 3c68419d48586986e8ff46d94ded6279
✓ Encontrado: l

--- Byte 4/128 ---
Buscando byte 3, usando 12 A's, comparando bloque 0
Target block: 1b38f17107fe807a1637302b7718098d
✓ Encontrado:  

--- Byte 5/128 ---
Buscando byte 4, usando 11 A's, comparando bloque 0
Target block: 3a0d35bf757feed824bd9a3dab9c224d
✓ Encontrado: G

--- Byte 6/128 ---
Buscando byte 5, usando 10 A's, comparando bloque 0
Target block: 8d70130874a13f21182419883fa67cd1
✓ Encontrado: o

--- Byte 7/128 ---
Buscando byte 6, usando 9 A's, comparando bloque 0
Target block: ac95fc863a5993ab18a3fe3fc48b1a55
✓ Encontrado: d

--- Byte 8/128 ---
Buscando byte 7, usando 8 A's, compa

In [69]:
secret_extended = recover_full_secret(YOUR_EMAIL, 144)
print(secret_extended.decode('utf-8', errors='replace'))


--- Byte 1/144 ---
Buscando byte 0, usando 15 A's, comparando bloque 0
Target block: fff9183484a7627c68cd52682af55633
✓ Encontrado: A

--- Byte 2/144 ---
Buscando byte 1, usando 14 A's, comparando bloque 0
Target block: 52ce51a5d82dbcfd0e4025c6c1257e4c
✓ Encontrado: l

--- Byte 3/144 ---
Buscando byte 2, usando 13 A's, comparando bloque 0
Target block: 3c68419d48586986e8ff46d94ded6279
✓ Encontrado: l

--- Byte 4/144 ---
Buscando byte 3, usando 12 A's, comparando bloque 0
Target block: 1b38f17107fe807a1637302b7718098d
✓ Encontrado:  

--- Byte 5/144 ---
Buscando byte 4, usando 11 A's, comparando bloque 0
Target block: 3a0d35bf757feed824bd9a3dab9c224d
✓ Encontrado: G

--- Byte 6/144 ---
Buscando byte 5, usando 10 A's, comparando bloque 0
Target block: 8d70130874a13f21182419883fa67cd1
✓ Encontrado: o

--- Byte 7/144 ---
Buscando byte 6, usando 9 A's, comparando bloque 0
Target block: ac95fc863a5993ab18a3fe3fc48b1a55
✓ Encontrado: d

--- Byte 8/144 ---
Buscando byte 7, usando 8 A's, compa

In [73]:
print(secret_extended)
secret_text = secret_extended.decode('utf-8')
print(secret_text)

b'All God\'s children are not beautiful.  Most of God\'s children are, in fact,\nbarely presentable.\n\t\t-- Fran Lebowitz, "Metropolitan Life"\x01'
All God's children are not beautiful.  Most of God's children are, in fact,
barely presentable.
		-- Fran Lebowitz, "Metropolitan Life"


In [81]:
secret_clean = secret_extended[:-1]
print(f"Secreto limpio: {secret_clean.decode('utf-8')}")

# Codificar en base64
secret_b64 = base64.b64encode(secret_clean).decode()

# Enviar
response = requests.post(
    answer_url,
    files={'message': secret_clean}
)

print(f"Respuesta: {response.text}")

Secreto limpio: All God's children are not beautiful.  Most of God's children are, in fact,
barely presentable.
		-- Fran Lebowitz, "Metropolitan Life"
Respuesta: ¡Ganaste!

